# DoubleGlovo: Analysis v2

## Analysis

### Imports

In [1]:
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append('c:\\Users\\Jordi Cremades\\Documents\\Repository')

from Utilities import query_engines

In [1]:
# Import dataset
# Understand the logic of the data set
# Get incremental orders number as showed by David. A DG Order has 78% extra orders in the following days... 
# Descartem els 0? Que passa si fem els dos escenaris
# Fer part A i B a la vegada --> Differents groups per a comparar incrementality de DG vs no DG

### Load Data

In [2]:

# Load data
df = pd.read_csv('inputs/dg_orders_1_x_user_access.csv')

# Flag treatment when order store accessed is DG
df['treatment'] =  np.where(df['store_accessed_channel']=='DoubleGlovo', True, False)

df = df[[
    'p_creation_date'
    ,'order_id'
    ,'store_accessed_channel'
    ,'treatment'
    ,'user_is_GNC'
    ,'user_is_prime'
    ,'months_old'
    ,'country'
    ,'n_orders_groceries_next28d'
    ,'aov_groceries_past30d'
]].copy()

# Drop duplicates
df_with_duplicates = df.copy()
df.drop_duplicates(subset='order_id',keep=False,inplace=True)

df

,p_creation_date,order_id,store_accessed_channel,treatment,user_is_GNC,user_is_prime,months_old,country,n_orders_groceries_next28d,aov_groceries_past30d
0,2023-09-30,706580245,NotDoubleGlovo,False,False,False,4,PT,2,24.02500
1,2023-09-04,690338612,NotDoubleGlovo,False,False,True,45,PT,4,18.54000
2,2023-08-24,682853212,NotDoubleGlovo,False,True,False,15,KE,0,0.00000
3,2023-09-04,690450489,NotDoubleGlovo,False,True,False,58,GE,0,0.00000
4,2023-09-30,706755438,NotDoubleGlovo,False,False,False,51,UA,0,0.00000
...,...,...,...,...,...,...,...,...,...,...
1812463,2023-08-20,680640457,NotDoubleGlovo,False,True,False,18,MA,0,0.00000
1812464,2023-08-26,684676536,NotDoubleGlovo,False,False,True,44,GE,0,10.88169
1812465,2023-08-20,680634979,NotDoubleGlovo,False,False,False,49,RO,0,0.00000
1812466,2023-08-20,680774904,NotDoubleGlovo,False,True,False,48,PL,0,0.00000


In [9]:
# Summary df
print('There are {} rows in the dataframe'.format(len(df)))
print('There are {} distinct orders'.format(df['order_id'].nunique()))
print('Min Date = {}'.format(df['p_creation_date'].min()))
print('Max Date = {}'.format(df['p_creation_date'].max()))
df['p_creation_date'] = pd.to_datetime(df['p_creation_date'])
difference_days = df['p_creation_date'].max()-df['p_creation_date'].min()
print('Difference days: {}'.format(difference_days.days))

There are 1567672 rows in the dataframe
There are 1567672 distinct orders
Min Date = 2023-07-13
Max Date = 2023-10-12
Difference days: 91


### Replicate GLM without duplicates

In [6]:
# Poisson GLM without duplicates
mod = smf.glm(formula="n_orders_groceries_next28d~treatment * user_is_GNC + user_is_prime + treatment:user_is_prime+np.sqrt(aov_groceries_past30d)+np.sqrt(months_old) + country", data=df, family=sm.families.Poisson()).fit()
print(mod.summary())

                     Generalized Linear Model Regression Results                      
Dep. Variable:     n_orders_groceries_next28d   No. Observations:              1567672
Model:                                    GLM   Df Residuals:                  1567644
Model Family:                         Poisson   Df Model:                           27
Link Function:                            Log   Scale:                          1.0000
Method:                                  IRLS   Log-Likelihood:            -2.2568e+06
Date:                        Tue, 05 Dec 2023   Deviance:                   3.1478e+06
Time:                                17:56:09   Pearson chi2:                 5.26e+06
No. Iterations:                             7   Pseudo R-squ. (CS):             0.5802
Covariance Type:                    nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
----------------------

In [7]:
# Poisson GLM with duplicates
mod = smf.glm(formula="n_orders_groceries_next28d~treatment * user_is_GNC + user_is_prime + treatment:user_is_prime+np.sqrt(aov_groceries_past30d)+np.sqrt(months_old) + country", data=df_with_duplicates, family=sm.families.Poisson()).fit()
print(mod.summary())

c:\Users\Jordi Cremades\Documents\Repository\.venv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Jordi Cremades\Documents\Repository\.venv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


                     Generalized Linear Model Regression Results                      
Dep. Variable:     n_orders_groceries_next28d   No. Observations:              1812456
Model:                                    GLM   Df Residuals:                  1812428
Model Family:                         Poisson   Df Model:                           27
Link Function:                            Log   Scale:                          1.0000
Method:                                  IRLS   Log-Likelihood:            -3.8409e+06
Date:                        Tue, 05 Dec 2023   Deviance:                   5.7363e+06
Time:                                17:57:10   Pearson chi2:                 1.94e+07
No. Iterations:                             7   Pseudo R-squ. (CS):             0.7803
Covariance Type:                    nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
----------------------

In [55]:
# Poisson GLM without duplicates
mod = smf.glm(formula="n_orders_groceries_next28d~treatment + country", data=df[df['user_is_GNC']==True], family=sm.families.Poisson()).fit()
print(mod.summary())

                     Generalized Linear Model Regression Results                      
Dep. Variable:     n_orders_groceries_next28d   No. Observations:               575097
Model:                                    GLM   Df Residuals:                   575075
Model Family:                         Poisson   Df Model:                           21
Link Function:                            Log   Scale:                          1.0000
Method:                                  IRLS   Log-Likelihood:            -3.8245e+05
Date:                        Tue, 05 Dec 2023   Deviance:                   5.8886e+05
Time:                                19:00:57   Pearson chi2:                 1.65e+06
No. Iterations:                             6   Pseudo R-squ. (CS):           0.005715
Covariance Type:                    nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------

### Segmentation

In [30]:
df.head()

,p_creation_date,order_id,store_accessed_channel,treatment,user_is_GNC,user_is_prime,months_old,country,n_orders_groceries_next28d,aov_groceries_past30d
0,2023-09-30,706580245,NotDoubleGlovo,False,False,False,4,PT,2,24.025
1,2023-09-04,690338612,NotDoubleGlovo,False,False,True,45,PT,4,18.540
2,2023-08-24,682853212,NotDoubleGlovo,False,True,False,15,KE,0,0.000
3,2023-09-04,690450489,NotDoubleGlovo,False,True,False,58,GE,0,0.000
4,2023-09-30,706755438,NotDoubleGlovo,False,False,False,51,UA,0,0.000


In [46]:
# NC per Country
cond = df['user_is_GNC'] == True

dfg = df[cond].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id        relative_difference
user_is_GNC         False      True          False   True                    
country                                                                      
AM               0.251106  0.550000         2035.0   20.0            1.190313
BA               0.146226  0.137931         2120.0   29.0           -0.056730
CI               0.328434  0.829268         6872.0   41.0            1.524914
ES               0.202758  0.540835       130752.0  551.0            1.667392
GE               0.216538  0.722772        21188.0  101.0            2.337859
GH               0.248100  0.230769         3289.0   65.0           -0.069853
HR               0.155037  0.286486        11236.0  185.0            0.847854
IT               0.216663  0.562500        61404.0  192.0            1.596193
KE               0.301628  0.652174        12164.0   46.0            1.162181
KG               0.185620  0.236559         7149.0  186.0            0.274424
KZ               0.211905  0.536424        14431.0  151.0            1.531436
MA               0.272616  0.662222        36865.0  225.0            1.429137
ME               0.277205  0.593750         2619.0   32.0            1.141916
NG               0.315086  0.690323         8750.0  155.0            1.190904
PL               0.247820  0.429379        63651.0  354.0            0.732622
PT               0.217826  0.512048        42534.0  166.0            1.350724
RO               0.204727  0.414062        63592.0  128.0            1.022510
RS               0.193840  0.272727        11494.0   11.0            0.406969
SI               0.149770  0.214286          868.0   14.0            0.430769
UA               0.240957  0.466599        66215.0  988.0            0.936438
UG               0.263347       NaN         2229.0    NaN                 NaN

In [50]:
pivot_table.describe()

mean_n_orders            count_order_id              \
user_is_GNC         False       True          False        True   
count           21.000000  20.000000      21.000000   20.000000   
mean             0.230819   0.477056   27212.238095  182.000000   
std              0.051468   0.194245   33342.418655  230.612685   
min              0.146226   0.137931     868.000000   11.000000   
25%              0.202758   0.283047    3289.000000   38.750000   
50%              0.217826   0.524236   11494.000000  139.500000   
75%              0.263347   0.608356   42534.000000  187.500000   
max              0.328434   0.829268  130752.000000  988.000000   

            relative_difference  
user_is_GNC                      
count                 20.000000  
mean                   1.032399  
std                    0.612011  
min                   -0.069853  
25%                    0.657158  
50%                    1.152049  
75%                    1.453081  
max                    2.337859

In [44]:
pivot_table.columns

MultiIndex([(      'mean_n_orders', False),
            (      'mean_n_orders',  True),
            (     'count_order_id', False),
            (     'count_order_id',  True),
            ('relative_difference',    '')],
           names=[None, 'user_is_GNC'])

In [35]:
# RC per Country
cond = df['user_is_GNC'] == False

dfg = df[cond].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id         relative_difference
user_is_GNC         False      True          False    True                    
country                                                                       
AM               1.118318  2.400000         1403.0    15.0            1.146080
BA               0.773151  1.166667         1609.0    12.0            0.508976
CI               1.607305  3.383333        15223.0   120.0            1.104973
ES               1.266479  3.516611       261364.0  1806.0            1.776684
GE               1.483864  3.625899        39538.0   139.0            1.443553
GH               1.005378  2.000000         3719.0    37.0            0.989302
HR               0.838572  1.745946        14477.0   185.0            1.082048
IT               1.024554  2.948187        89516.0   386.0            1.877531
KE               1.691369  4.940171        25166.0   117.0            1.920812
KG               1.252462  2.785714         5787.0    56.0            1.224190
KZ               1.313547  2.843373        16594.0   166.0            1.164653
MA               1.634348  4.259021        78974.0   776.0            1.605945
ME               1.475279  3.540000         4652.0    50.0            1.399545
NG               1.366978  2.073529         6093.0    68.0            0.516871
PL               1.283223  2.753927       108473.0   382.0            1.146102
PT               1.279526  3.888889        67550.0   459.0            2.039319
RO               1.715795  4.120455       122190.0   440.0            1.401484
RS               1.231696  3.285714        19422.0    14.0            1.667634
SI               0.756874  1.285714          691.0     7.0            0.698716
UA               1.426337  2.549608       101720.0   766.0            0.787522
UG               1.059262       NaN         2413.0     NaN                 NaN

In [36]:
# Prime per Country
cond = df['user_is_prime'] == True

dfg = df[cond].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id         relative_difference
user_is_GNC         False      True          False    True                    
country                                                                       
AM               1.218143  2.583333          463.0    12.0            1.120715
BA               0.333333  0.000000            6.0     1.0           -1.000000
CI               2.794237  4.388889         1978.0    54.0            0.570693
ES               1.829496  3.494186        74186.0  1376.0            0.909917
GE               2.404633  3.243697        12043.0   119.0            0.348936
GH               0.700000       NaN           10.0     NaN                 NaN
HR               0.315789  7.000000           19.0     2.0           21.166667
IT               1.391506  2.694915        16858.0   236.0            0.936690
KE               3.296838  5.654545         2119.0    55.0            0.715142
KG               1.974593  2.911111         1535.0    45.0            0.474284
KZ               1.581563  2.246073         6259.0   191.0            0.420161
MA               2.274702  4.253401        26840.0   588.0            0.869872
ME               1.076923       NaN           39.0     NaN                 NaN
NG               0.333333       NaN            6.0     NaN                 NaN
PL               2.083374  3.745763         8264.0    59.0            0.797931
PT               2.127280  3.753541        16012.0   353.0            0.764479
RO               3.086180  4.274011        38292.0   354.0            0.384887
RS               1.959074  3.692308         4667.0    13.0            0.884721
SI               1.750000       NaN            4.0     NaN                 NaN
UA               2.000000  0.000000           51.0     1.0           -1.000000
UG               2.333333       NaN            3.0     NaN                 NaN

In [37]:
# No Prime per Country
cond = df['user_is_prime'] == False

dfg = df[cond].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id         relative_difference
user_is_GNC         False      True          False    True                    
country                                                                       
AM               0.509580  0.695652         2975.0    23.0            0.365149
BA               0.416868  0.450000         3723.0    40.0            0.079478
CI               1.053736  1.897196        20117.0   107.0            0.800448
ES               0.697638  1.876656       317930.0   981.0            1.690015
GE               0.704517  1.578512        48683.0   121.0            1.240560
GH               0.649900  0.872549         6998.0   102.0            0.342590
HR               0.540048  0.983696        25694.0   368.0            0.821496
IT               0.608375  1.783626       134062.0   342.0            1.931786
KE               1.114652  2.750000        35211.0   108.0            1.467138
KG               0.486273  0.350254        11401.0   197.0           -0.279718
KZ               0.603892  0.984127        24766.0   126.0            0.629640
MA               0.877178  2.307506        88999.0   413.0            1.630601
ME               1.043556  2.390244         7232.0    82.0            1.290479
NG               0.747051  1.112108        14837.0   223.0            0.488663
PL               0.840669  1.451994       163860.0   677.0            0.727189
PT               0.655190  2.003676        94072.0   272.0            2.058163
RO               0.708495  1.649533       147490.0   214.0            1.328219
RS               0.647910  0.083333        26249.0    12.0           -0.871381
SI               0.415434  0.571429         1555.0    21.0            0.375498
UA               0.958638  1.377068       167884.0  1753.0            0.436484
UG               0.676008       NaN         4639.0     NaN                 NaN

In [39]:
# Prime and NC per Country
cond = df['user_is_GNC'] == True
cond2 = df['user_is_prime'] == True

dfg = df[cond & cond2].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id        relative_difference
user_is_GNC         False      True          False   True                    
country                                                                      
AM               0.523529  0.750000          170.0    4.0            0.432584
BA               0.000000  0.000000            1.0    1.0                 NaN
CI               0.770186  1.166667          161.0    6.0            0.514785
ES               0.335638  0.696335         9546.0  191.0            1.074661
GE               0.487027  0.827586         1349.0   29.0            0.699260
GH               0.000000       NaN            3.0    NaN                 NaN
HR               0.000000       NaN            2.0    NaN                 NaN
IT               0.339089  0.977778         3533.0   45.0            1.883547
KE               0.938547  0.333333          179.0    6.0           -0.644841
KG               0.491857  0.818182          307.0   11.0            0.663456
KZ               0.356669  0.602740         1657.0   73.0            0.689915
MA               0.479207  0.970149         3078.0   67.0            1.024488
ME               0.000000       NaN            2.0    NaN                 NaN
NG               0.000000       NaN            1.0    NaN                 NaN
PL               0.441071  0.363636         1120.0   11.0           -0.175561
PT               0.363263  0.812500         2194.0   48.0            1.236669
RO               0.469090  0.685714         3494.0   35.0            0.461797
RS               0.338362  1.000000          464.0    2.0            1.955414
SI               0.000000       NaN            1.0    NaN                 NaN
UA               0.166667       NaN            6.0    NaN                 NaN
UG               1.000000       NaN            1.0    NaN                 NaN

In [43]:
# No Prime and NC per Country
cond = df['user_is_GNC'] == True
cond2 = df['user_is_prime'] == False

dfg = df[cond & cond2].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id        relative_difference
user_is_GNC         False      True          False   True                    
country                                                                      
AM               0.226273  0.500000         1865.0   16.0            1.209716
BA               0.146295  0.142857         2119.0   28.0           -0.023502
CI               0.317836  0.771429         6711.0   35.0            1.427125
ES               0.192292  0.458333       121206.0  360.0            1.383522
GE               0.198145  0.680556        19839.0   72.0            2.434633
GH               0.248326  0.230769         3286.0   65.0           -0.070701
HR               0.155065  0.286486        11234.0  185.0            0.847525
IT               0.209189  0.435374        57871.0  147.0            1.081244
KE               0.292115  0.700000        11985.0   40.0            1.396315
KG               0.171880  0.200000         6842.0  175.0            0.163605
KZ               0.193127  0.474359        12774.0   78.0            1.456207
MA               0.253796  0.531646        33787.0  158.0            1.094777
ME               0.277417  0.593750         2617.0   32.0            1.140281
NG               0.315122  0.690323         8749.0  155.0            1.190654
PL               0.244359  0.431487        62531.0  343.0            0.765792
PT               0.209916  0.389831        40340.0  118.0            0.857081
RO               0.189357  0.311828        60098.0   93.0            0.646769
RS               0.187761  0.111111        11030.0    9.0           -0.408230
SI               0.149942  0.214286          867.0   14.0            0.429121
UA               0.240964  0.466599        66209.0  988.0            0.936384
UG               0.263016       NaN         2228.0    NaN                 NaN

In [41]:
# Prime and NC per Country
cond = df['user_is_GNC'] == False
cond2 = df['user_is_prime'] == True

dfg = df[cond & cond2].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id         relative_difference
user_is_GNC         False      True          False    True                    
country                                                                       
AM               1.621160  3.500000          293.0     8.0            1.158947
BA               0.400000       NaN            5.0     NaN                 NaN
CI               2.973583  4.791667         1817.0    48.0            0.611412
ES               2.050108  3.945148        64640.0  1185.0            0.924361
GE               2.646531  4.022222        10694.0    90.0            0.519809
GH               1.000000       NaN            7.0     NaN                 NaN
HR               0.352941  7.000000           17.0     2.0           18.833333
IT               1.670544  3.099476        13325.0   191.0            0.855369
KE               3.514433  6.306122         1940.0    49.0            0.794350
KG               2.345277  3.588235         1228.0    34.0            0.529984
KZ               2.022599  3.262712         4602.0   118.0            0.613128
MA               2.507281  4.675624        23762.0   521.0            0.864819
ME               1.135135       NaN           37.0     NaN                 NaN
NG               0.400000       NaN            5.0     NaN                 NaN
PL               2.340845  4.520833         7144.0    48.0            0.931282
PT               2.407367  4.216393        13818.0   305.0            0.751454
RO               3.348957  4.667712        34798.0   319.0            0.393781
RS               2.137997  4.181818         4203.0    11.0            0.955952
SI               2.333333       NaN            3.0     NaN                 NaN
UA               2.244444  0.000000           45.0     1.0           -1.000000
UG               3.000000       NaN            2.0     NaN                 NaN

In [42]:
# No Prime and NC per Country
cond = df['user_is_GNC'] == False
cond2 = df['user_is_prime'] == False

dfg = df[cond & cond2].groupby(['country', 'treatment']).agg({'n_orders_groceries_next28d': 'mean', 'order_id': 'count'}).reset_index()
dfg.columns = ['country', 'user_is_GNC', 'mean_n_orders', 'count_order_id']
pivot_table = dfg.pivot(index='country', columns='user_is_GNC', values=['mean_n_orders', 'count_order_id'])
pivot_table['relative_difference'] = (pivot_table[('mean_n_orders', True)] - pivot_table[('mean_n_orders', False)]) / pivot_table[('mean_n_orders', False)]
pivot_table

mean_n_orders           count_order_id        relative_difference
user_is_GNC         False      True          False   True                    
country                                                                      
AM               0.985586  1.142857         1110.0    7.0            0.159572
BA               0.774314  1.166667         1604.0   12.0            0.506710
CI               1.422124  2.444444        13406.0   72.0            0.718868
ES               1.008992  2.698873       196724.0  621.0            1.674820
GE               1.052801  2.897959        28844.0   49.0            1.752617
GH               1.005388  2.000000         3712.0   37.0            0.989282
HR               0.839142  1.688525        14460.0  183.0            1.012203
IT               0.911577  2.800000        76191.0  195.0            2.071598
KE               1.539094  3.955882        23226.0   68.0            1.570267
KG               0.958105  1.545455         4559.0   22.0            0.613033
KZ               1.041444  1.812500        11992.0   48.0            0.740372
MA               1.258658  3.407843        55212.0  255.0            1.707522
ME               1.478007  3.540000         4615.0   50.0            1.395118
NG               1.367773  2.073529         6088.0   68.0            0.515990
PL               1.208657  2.500000       101329.0  334.0            1.068412
PT               0.989485  3.240260        53732.0  154.0            2.274694
RO               1.065498  2.677686        87392.0  121.0            1.513084
RS               0.981405  0.000000        15219.0    3.0           -1.000000
SI               0.750000  1.285714          688.0    7.0            0.714286
UA               1.425975  2.552941       101675.0  765.0            0.790313
UG               1.057652       NaN         2411.0    NaN                 NaN